# Trabalho 1 - Análise de Portifólios com Fronteira de Markowitz

**Aluno:** Luiz Fernando Rabelo (11796893)

## Bibliotecas Utilizadas

Para a resolução do trabalho, foram utilizadas as bibliotecas yfinance (Yahoo Finance), numpy, pandas e matplotlib, as quais são importadas abaixo:

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Ativos Considerados

Para a análise do portifólio, foram consideradas as seguintes ações preferenciais, no período de 1 de janeiro de 2015 à 1 de janeiro de 2023:

- PETR4 (Petrobras);
- POMO4 (Marcopolo).
- CMIG4 (CEMIG);

In [ ]:
acoes = ['PETR4.SA', 'POMO4.SA', 'CMIG4.SA']
dados = yf.download(acoes, '2015-01-01', '2023-01-01', '1d')['Adj Close']

## Normalização dos dados

Para que a comparação entre os dados seja feita de forma mais direta, é importante convertê-los para a mesma ordem de grandeza.

In [ ]:
dados = dados * 100 / dados.iloc[0]

## Visualização da Evolução das Ações

Os dados podem ser visualizados tanto em tabela como em gráfico.

In [ ]:
display(dados)
dados.plot(figsize=(15,5));

## Cômputo da Correlação Entre os Dados

A fim de detectar como se deu a variação conjunta dos ativos, calculamos a correlação entre os dados.

In [ ]:
mudanca_dados = dados.pct_change()          # variações
mudanca_dados = mudanca_dados.fillna(0)     # sem dados => variação 0

mudanca_dados.corr()

## Cômputo da Volatilidade e Retorno dos Ativos Individuais

Podemos calcular agora o retorno e a volatilidade de cada ativo.

In [ ]:
retorno_acumulado = (dados.iloc[dados.count()[0] - 1] / dados.iloc[0]) - 1
print("Retorno acumulado:\n", retorno_acumulado, sep='')

In [ ]:
retorno_anualizado = ((dados.iloc[-1] / dados.iloc[0]) ** (1/5)) - 1
print("Retorno anualizado:\n", retorno_anualizado, sep='')

In [ ]:
volatilidade_anualizada = mudanca_dados.std() * np.sqrt(252)
print("Volatilidade anualizada:\n", volatilidade_anualizada, sep='')

## Função Para Calcular de Retorno e Volatilidade em Portifólio

Podemos definir uma função que determina o retorno e a volatividade de dado portifólio, conforme os pesos passados por parâmetro.

In [ ]:
def calcular_retorno_e_volatilidade(ativos, pesos):
    portifolio = ativos.dot(pesos)
    mudanca_portifolio = portifolio.pct_change()
    mudanca_portifolio = mudanca_portifolio.fillna(0)
    retorno = ((portifolio.iloc[-1] / portifolio.iloc[0]) ** (1/5)) - 1
    volatilidade = mudanca_portifolio.std() * np.sqrt(252)
    return retorno, volatilidade

## Portifólio com 2 Ativos

Inicialmente, podemos compor um portifólio com os 2 ativos de menor correlação (PETR4 e POMO4).

In [ ]:
pontos = []
min_vol_ret = [100, 0]
pesos_otimos = [0, 0, 0]

for w in range(0, 101, 5):
    pesos_aplicados = [0, w/100, 1-w/100]
    ret, vol = calcular_retorno_e_volatilidade(dados, pesos_aplicados)
    print(f'Aloc: {pesos_aplicados[1]:.2f} x {pesos_aplicados[2]:.2f} Ret:{ret:.3f} Vol:{vol:.3f}') 
    pontos.append((ret, vol))
    if vol < min_vol_ret[0]:
        min_vol_ret = [vol, ret]
        pesos_otimos = pesos_aplicados[:]

É possível determinar a "composição ideal" dos ativos, com risco mínimo e rentabilidade correspondente:

In [ ]:
print(pesos_otimos)
print(min_vol_ret)

Exibição dos pontos:

In [ ]:
lp = np.array(pontos).T

plt.scatter(lp[[1][:]],lp[[0][:]]);

plt.ylabel("Retorno");
plt.xlabel("Volatilidade");

plt.scatter(volatilidade_anualizada['POMO4.SA'], retorno_anualizado['POMO4.SA'], color='red');
plt.text(volatilidade_anualizada['POMO4.SA'], retorno_anualizado['POMO4.SA'], 'POMO4');

plt.scatter(volatilidade_anualizada['PETR4.SA'], retorno_anualizado['PETR4.SA'], color='red');
plt.text(volatilidade_anualizada['PETR4.SA'], retorno_anualizado['PETR4.SA'], 'PETR4');

plt.scatter(min_vol_ret[0], min_vol_ret[1], color='green');
plt.text(min_vol_ret[0], min_vol_ret[1], 'Min. Vol.');

Com a composição do portifólio, usamos os "pesos ótimos" (de menor volatilidade) para calcularmos o retorno e a volatilidade.

In [ ]:
dados['PORT1'] = dados.dot(pesos_otimos)

mudanca_dados = dados.pct_change()
mudanca_dados = mudanca_dados.fillna(0)

In [ ]:
retorno_anualizado = ((dados.iloc[-1] / dados.iloc[0]) ** (1/5)) - 1
print("Retorno anualizado:\n", retorno_anualizado, sep='')

In [ ]:
volatilidade_anualizada = mudanca_dados.std() * np.sqrt(252)
print("Volatilidade anualizada:\n", volatilidade_anualizada, sep='')

É possível também determinar o Drawdown, o qual pode ser visualizado por tabela ou gráfico:

In [ ]:
draw_down = pd.DataFrame()

for ativo in dados.columns:
    linha = []
    for i in range(dados.count()[0]):
        linha.append((dados[ativo].iloc[i] / dados[ativo].iloc[:i+1].max() - 1) * 100)
    draw_down[ativo] = linha

draw_down['Data'] = dados.index.values
draw_down.set_index(keys='Data', inplace=True)

print(draw_down[['PETR4.SA', 'POMO4.SA', 'PORT1']].min())
display(draw_down[['PETR4.SA', 'POMO4.SA', 'PORT1']])
draw_down[['PETR4.SA', 'POMO4.SA', 'PORT1']].plot(figsize=(15,5));

## Portifólio com 3 Ativos

Retiramos a coluna do portifólio recém montado de maneira temporária do dataframe dos dados para facilitar a multiplicação pelos pesos (questão de ordem matricial).

In [ ]:
dados = dados.drop(['PORT1'], axis=1)
mudanca_dados = mudanca_dados.drop(['PORT1'], axis=1)

Para o novo portifólio, serão considerados os 3 ativos: CMIG4, PETR4 e POMO4:

In [ ]:
pontos = []
min_vol_ret = [100, 0]
pesos_otimos = [0, 0, 0]
for w1 in range(0, 101, 5):
    for w2 in range(0, 101 - w1, 5):
        pesos_aplicados = [w1/100, w2/100, 1 - w1/100 - w2/100]
        ret, vol = calcular_retorno_e_volatilidade(dados, pesos_aplicados)
        print(f'Aloc:{pesos_aplicados[0]:.2f} x {pesos_aplicados[1]:.2f} {pesos_aplicados[2]:.2f} Ret:{ret:.3f} Vol:{vol:.3f}')
        pontos.append((ret, vol))
        if vol < min_vol_ret[0]:
            min_vol_ret = [vol, ret]
            pesos_otimos = pesos_aplicados[:]

Composição ideal dos ativos, com risco mínimo e rentabilidade correspondente:

In [ ]:
print(pesos_otimos)
print(min_vol_ret)

Exibição dos pontos:

In [ ]:
lp = np.array(pontos).T
plt.scatter(lp[[1][:]],lp[[0][:]]);

plt.ylabel("Retorno");
plt.xlabel("Volatilidade");

plt.scatter(volatilidade_anualizada['CMIG4.SA'], retorno_anualizado['CMIG4.SA'], color='red');
plt.text(volatilidade_anualizada['CMIG4.SA'], retorno_anualizado['CMIG4.SA'], 'CMIG4');

plt.scatter(volatilidade_anualizada['PETR4.SA'], retorno_anualizado['PETR4.SA'], color='red');
plt.text(volatilidade_anualizada['PETR4.SA'], retorno_anualizado['PETR4.SA'], 'PETR4');

plt.scatter(volatilidade_anualizada['POMO4.SA'], retorno_anualizado['POMO4.SA'], color='red');
plt.text(volatilidade_anualizada['POMO4.SA'], retorno_anualizado['POMO4.SA'], 'POMO4');

plt.scatter(min_vol_ret[0], min_vol_ret[1], color='green');
plt.text(min_vol_ret[0], min_vol_ret[1], 'Min. Vol.');

Com essa outra composição do portifólio, usando os novos pesos de menor volatilidade, calculamos novamente o retorno e a rentabilidade:

In [ ]:
dados['PORT1'] = dados.dot(pesos_otimos)

mudanca_dados = (dados - dados.shift(1)) / dados.shift(1)
mudanca_dados = mudanca_dados.fillna(0)

In [ ]:
retorno_anualizado = ((dados.iloc[-1] / dados.iloc[0]) ** (1/5)) - 1
print("Retorno anualizado:\n", retorno_anualizado, sep='')

In [ ]:
volatilidade_anualizada = mudanca_dados.std() * np.sqrt(252)
print("Volatilidade anualizada:\n", volatilidade_anualizada, sep='')

Cálculo do drawdown:

In [ ]:
draw_down = pd.DataFrame()

for ativo in dados.columns:
    linha = []
    for i in range(dados.count()[0]):
        linha.append((dados[ativo].iloc[i] / dados[ativo].iloc[:i+1].max() - 1) * 100)
    draw_down[ativo] = linha

draw_down['Data'] = dados.index.values
draw_down.set_index(keys='Data', inplace=True)

print(draw_down.min())
display(draw_down)
draw_down.plot(figsize=(15,5));